# CITE-seq prediction with the top problem from OpenProblems 2022 competition

In this repository, we provide a reimplementation of the best CITE-seq model from OpenProblems modality prediction competition in 2022. Original code can be found on [GitHub](https://github.com/senkin13/kaggle/blob/master/Open-Problems-Multimodal-Single-Cell-Integration-2nd-Place-Solution/senkin13/preprocess_cite.ipynb) and on [Kaggle](https://www.kaggle.com/code/senkin13/2nd-place-gru-cite). Here, we adapt this code to work with mudata format and modern library versions. We also make it more efficient by using power of `muon` and `scanpy` libraries.

Let's first import some libraries:

In [ ]:
import logging

import mudata as mu
import numpy as np
import scanpy as sc
import pooch  # To download mudata from figshare, feel free to drop if manually downloaded

from senkin_tmp_cite_pred.preprocess import preprocess_data

/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Create directories for data and models:

In [2]:
%%bash

mkdir -p data/
mkdir -p models/

Let's download data from 2021 competition to make sure that the code runs for another dataset:

In [4]:
pooch.retrieve(
    url="https://figshare.com/ndownloader/files/41452287",  # Feel free to manually download the file to avoid installing pooch
    fname="data/cite_filtered.h5mu",
    path=".",
    known_hash=None,
    progressbar=True,
)

mdata = mu.read_h5mu("data/cite_filtered.h5mu")
mdata

100%|█████████████████████████████████████| 1.55G/1.55G [00:00<00:00, 2.30TB/s]
SHA256 hash of downloaded file: 2b11fe2ac8cea96cf83992c2ab231f3474d2173ccec49fc6b7b2499ad213d7f4
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities b

MuData object with n_obs × n_vars = 122016 × 36741
  var:	'gene_ids', 'feature_types'
  2 modalities
    rna:	122016 x 36601
      obs:	'donor', 'batch'
      var:	'gene_ids', 'feature_types'
    prot:	122016 x 140
      obs:	'donor', 'batch'
      var:	'gene_ids', 'feature_types'

If the data was downloaded previously, comment out the cell above and run:

In [5]:
mdata = mu.read_h5mu("data/cite_filtered.h5mu")
mdata

/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


MuData object with n_obs × n_vars = 122016 × 36741
  var:	'gene_ids', 'feature_types'
  2 modalities
    rna:	122016 x 36601
      obs:	'donor', 'batch'
      var:	'gene_ids', 'feature_types'
    prot:	122016 x 140
      obs:	'donor', 'batch'
      var:	'gene_ids', 'feature_types'

Let's select a random subset of cells to test the code:

In [6]:
random_cells = np.random.choice(mdata.obs_names, size=10000, replace=False)
random_cells

array(['GGGCTACTCGAGATGG-1-1-0-0-0-0-0-0-0-0',
       'TGTTACTGTCAATCTG-1-1-0-0-0', 'TCGGGCACAAATTGGA-1-1-0-0-0-0', ...,
       'TCCGGGAAGTAACAGT-1-1-0-0', 'TCTCTGGCAGGCAATG-1-1-0',
       'CTCAACCAGGGCAGTT-1-1-0-0-0-0-0-0-0-0-0-0'],
      shape=(10000,), dtype=object)

In [7]:
mdata = mdata[random_cells].copy()

/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [8]:
adata_rna = mdata.mod["rna"]
adata_prot = mdata.mod["prot"]

In the 2022 competition, the batch effect correction for this approach was done per day. Let's follow that, and extract day in a separate `.obs` column:

In [9]:
adata_rna.obs["day"] = adata_rna.obs["donor"].str[-2:]

In [10]:
adata_rna.obs["day"].value_counts()

day
d1    3547
d7    1200
d6    1182
d5     951
d9     832
d3     658
d4     625
d2     613
d8     392
Name: count, dtype: int64

Optionally, we can add a set of known features, which are important for prediction of the given surface proteins. In the original solution, it was a set of genes that encode proteins in the CITE-seq data. In this example notebook, we'll simply extract genes that have identical names for proteins and RNA. Note that it will give an incomplete list:

In [11]:
known_features = adata_prot.var["gene_ids"][adata_prot.var["gene_ids"].isin(adata_rna.var_names)]
print(f"Number of proteins with identical names in RNA: {len(known_features)}")

Number of proteins with identical names in RNA: 37


Now we can perform required preprocessing with just one function. It includes:
- Removal of constant features
- 200 components TSVD of CLR-transformed data
- 100 components PCA of customly normalized data (see documentation of `senkin_normalize` for details)
- Selection of raw expression for known features and for genes correlated with target proteins
- DSB normalization of protein data. For this `empty_count_range` setting is needed. Here, we used the same values as in 2022 competition. But this is data dependent, so make sure to select a meaningful range!

Note that the nomber of cells will become smaller after this step, because some droplets will be used as a background for DSB normalization. Additionally, number of genes will decrease as constant genes will be removed from RNA data.

In [12]:
mdata = preprocess_data(
    mdata,
    empty_counts_range=(1.5, 2.8),
    batch_key="day",
    group_key="donor",
    known_features=known_features
)

/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)
/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/l

Note new `obsm` fields in RNA modality, and a `dsb` layer in the protein modality:

In [13]:
mdata

MuData object with n_obs × n_vars = 9774 × 36741
  var:	'gene_ids', 'feature_types'
  2 modalities
    rna:	9774 x 25156
      obs:	'donor', 'batch', 'day'
      var:	'gene_ids', 'feature_types'
      uns:	'log1p', 'selected_features'
      obsm:	'X_log_normalized', 'X_clr_tsvd', 'X_sqrt_norm', 'X_pca_sqrt_norm', 'X_raw_selected'
    prot:	9774 x 140
      obs:	'donor', 'batch'
      var:	'gene_ids', 'feature_types'
      layers:	'dsb'

Let's save preprocessed files to reuse them later:

In [14]:
mdata.mod["rna"].write_h5ad("data/cite_filtered_preprocessed_rna.h5ad")
mdata.mod["prot"].write_h5ad("data/cite_filtered_preprocessed_prot.h5ad")

# LGBM predictions

Next, we'll use preprocessed data to make initial prediction of target proteints with Light Gradient Bossting (LGBM) models. The dimensionality of the predictions will be further reduced with TSVD to 100 components. While these predictions are not used per se, they will become features for neural networks. As we showed in our analysis, this makes the resulting prediction much more accurate.

In [15]:
from senkin_tmp_cite_pred.lgbm_models import train_lightgbm_models
from sklearn.model_selection import KFold

In [16]:
adata_rna = sc.read_h5ad("data/cite_filtered_preprocessed_rna.h5ad")
adata_rna

AnnData object with n_obs × n_vars = 9774 × 25156
    obs: 'donor', 'batch', 'day'
    var: 'gene_ids', 'feature_types'
    uns: 'log1p', 'selected_features'
    obsm: 'X_clr_tsvd', 'X_log_normalized', 'X_pca_sqrt_norm', 'X_raw_selected', 'X_sqrt_norm'

In [17]:
adata_prot = sc.read("data/cite_filtered_preprocessed_prot.h5ad")
adata_prot

AnnData object with n_obs × n_vars = 9774 × 140
    obs: 'donor', 'batch'
    var: 'gene_ids', 'feature_types'
    layers: 'dsb'

In [18]:
adata_rna.obs["day"].value_counts()

day
d1    3443
d7    1187
d6    1168
d5     937
d9     803
d3     647
d2     600
d4     600
d8     389
Name: count, dtype: int64

If your data contains "train" and "test" subsets, feel free to use them. Otherwise, we need to define train-test split. Let's use days 6-8 as test here. For test dataset, we won't use ground truth target protein levels for training. Here, we have a ground truth to validate the quality of our results, but in a real life scenario it can be a dataset, for which you want to impute protein modality.

In [19]:
adata_rna.obs["split"] = adata_rna.obs["day"].apply(lambda x: "test" if x in ["d6", "d7", "d8"] else "train")

In [20]:
adata_rna.obs["split"].value_counts()

split
train    7030
test     2744
Name: count, dtype: int64

Additionally, we'll define cross-validation splits for the training dataset. It'll help us to see how well our models predict the data on an unseen fold. Similarly to the original approach, we'll use random split to 5 folds.

In [21]:
folds = KFold(n_splits= 5, shuffle=True, random_state=666)

We'll save which cell comes from each fold to make sure results are reproducible after the `folds` variable is lost:

In [22]:
for i, (train_idxs, test_idxs) in enumerate(folds.split(adata_rna)):
    adata_rna.obs[f"fold_{i}"] = "test"
    adata_rna.obs.loc[adata_rna.obs_names[train_idxs], f"fold_{i}"] = "train"

adata_rna.obs["fold_0"].value_counts()

fold_0
train    7819
test     1955
Name: count, dtype: int64

In [23]:
train_cell_ids = adata_rna.obs_names[adata_rna.obs["split"] == "train"]
test_cell_ids = adata_rna.obs_names[adata_rna.obs["split"] == "test"]

If you wish to see less logs, feel free to remove the cell below:

In [24]:
logging.basicConfig(level=logging.DEBUG)

Train LGBM models. 4 models will be trained, using different subsets of training data, and different preprocessing of the target proteins. In this example, we'll only build models for 3 proteins here, and train them for only 3 epochs. Remove `[:, 3]` after `adata_prot` to use all the proteins, and remove `num_boost_round` `early_stopping_rounds`, and `n_tsvd_components` to use the default parameters

In [25]:
adata_rna = train_lightgbm_models(
    adata_rna,
    adata_prot[:, :3],
    train_cell_ids=train_cell_ids,
    test_cell_ids=test_cell_ids,
    folds=folds,
    num_boost_round=3,
    early_stopping_rounds=2,
    n_tsvd_components=10
)

INFO:senkin_tmp_cite_pred.lgbm_models:Initializing arrays in obsm with zeros
INFO:senkin_tmp_cite_pred.lgbm_models:Training LightGBM models
INFO:senkin_tmp_cite_pred.lgbm_models:Training LightGBM model 1 for predicting DSB-normalized protein expression from log-normalized RNA expression
INFO:senkin_tmp_cite_pred.lgbm_models:Training LightGBM models for 3 targets
DEBUG:senkin_tmp_cite_pred.lgbm_models:Training LightGBM model for target 0
DEBUG:senkin_tmp_cite_pred.lgbm_models:Fold: 0
DEBUG:senkin_tmp_cite_pred.lgbm_models:Fold: 1


Training until validation scores don't improve for 2 rounds
Did not meet early stopping. Best iteration is:
[3]	valid_0's l2: 2.39419


DEBUG:senkin_tmp_cite_pred.lgbm_models:Fold: 2


Training until validation scores don't improve for 2 rounds
Did not meet early stopping. Best iteration is:
[3]	valid_0's l2: 2.60932


DEBUG:senkin_tmp_cite_pred.lgbm_models:Fold: 3


...

DEBUG:senkin_tmp_cite_pred.lgbm_models:Fold: 3


Training until validation scores don't improve for 2 rounds
Did not meet early stopping. Best iteration is:
[3]	valid_0's l2: 7072.69


DEBUG:senkin_tmp_cite_pred.lgbm_models:Fold: 4


Training until validation scores don't improve for 2 rounds
Did not meet early stopping. Best iteration is:
[3]	valid_0's l2: 2442.77
Training until validation scores don't improve for 2 rounds
Did not meet early stopping. Best iteration is:
[3]	valid_0's l2: 2707.63


/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/vladimir.shitov/miniconda3/envs/2025_08_citeseqpred/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
INFO:senkin_tmp_cite_pred.lgbm_models:CV score: nan
INFO:senkin_tmp_cite_pred.lgbm_models:TSVD-reducing predictions to 2 components


Note that we know have 4 additional `X_lgbm` arrays in the `.obsm`:

In [26]:
adata_rna

AnnData object with n_obs × n_vars = 9774 × 25156
    obs: 'donor', 'batch', 'day', 'split', 'fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4'
    var: 'gene_ids', 'feature_types'
    uns: 'log1p', 'selected_features'
    obsm: 'X_clr_tsvd', 'X_log_normalized', 'X_pca_sqrt_norm', 'X_raw_selected', 'X_sqrt_norm', 'X_lgbm_1', 'X_lgbm_2', 'X_lgbm_3', 'X_lgbm_4'

# Train neural networks

In [27]:
from senkin_tmp_cite_pred.nn_models import train_nn_models

Finally, we'll train two neural networks with originally proposed architectures to predict target proteins. First, let's prepare datasets:

In [28]:
train_cite_X = np.concatenate([
    adata_rna[train_cell_ids].obsm["X_clr_tsvd"],
    adata_rna[train_cell_ids].obsm["X_pca_sqrt_norm"],
    adata_rna[train_cell_ids].obsm["X_raw_selected"].toarray(),
    adata_rna[train_cell_ids].obsm["X_lgbm_1"],
    adata_rna[train_cell_ids].obsm["X_lgbm_2"],
    adata_rna[train_cell_ids].obsm["X_lgbm_3"],
    adata_rna[train_cell_ids].obsm["X_lgbm_4"],
], axis=1)

test_cite_X = np.concatenate([
    adata_rna[test_cell_ids].obsm["X_clr_tsvd"],
    adata_rna[test_cell_ids].obsm["X_pca_sqrt_norm"],
    adata_rna[test_cell_ids].obsm["X_raw_selected"].toarray(),
    adata_rna[test_cell_ids].obsm["X_lgbm_1"],
    adata_rna[test_cell_ids].obsm["X_lgbm_2"],
    adata_rna[test_cell_ids].obsm["X_lgbm_3"],
    adata_rna[test_cell_ids].obsm["X_lgbm_4"],
], axis=1)

train_cite_y = adata_prot[train_cell_ids].layers["dsb"]

In [29]:
print("Total number of features:", train_cite_X.shape[1])

Total number of features: 1039


This function will perform cross-validated training and prediction, and return us aggregated prediction of two models. For the purposes of the example, we'll on;y use 3 target proteins and 3 epochs to train the model. Remove `[:, :3]` from `train_cite_y` and set `EPOCHS` to 100 (or just leave the default) to predict all the proteins with the original parameters:

In [30]:
train_preds, test_preds = train_nn_models(
    train_cell_ids,
    train_cite_X,
    train_cite_y[:, :3],
    test_cell_ids,
    test_cite_X,
    folds, 
    EPOCHS=3
)

0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1039)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1039)   │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 3600)      │ 30,682,800 │ reshape[0][0]     │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gaussian_dropout    │ (None, 3600)      │          0 │ bidirectional[0]… │
│ (GaussianDropout)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1800)      │  6,481,800 │ gaussian_dropout… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gaussian_dropout_1  │ (None, 1800)      │          0 │ dense[0][0]       │
│ (GaussianDropout)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1800)      │  3,241,800 │ gaussian_dropout… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gaussian_dropout_2  │ (None, 1800)      │          0 │ dense_1[0][0]     │
│ (GaussianDropout)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 7200)      │          0 │ gaussian_dropout… │
│ (Concatenate)       │                   │            │ gaussian_dropout… │
│                     │                   │            │ gaussian_dropout… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 3)         │     21,603 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 40,428,003 (154.22 MB)

 Trainable params: 40,428,003 (154.22 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1039)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1500)           │     1,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_dropout                │ (None, 1500)           │             0 │
│ (GaussianDropout)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1500)           │     2,251,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_dropout_1              │ (None, 1500)           │             0 │
│ (GaussianDropout)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1500)           │     2,251,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_dropout_2              │ (None, 1500)           │             0 │
│ (GaussianDropout)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 1, 1500)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 1400)           │     9,248,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_dropout_3              │ (None, 1400)           │             0 │
│ (GaussianDropout)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │         4,203 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,315,603 (58.42 MB)

 Trainable params: 15,315,603 (58.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 257ms/step - loss: 9553.7773 - val_loss: 29.2939 - learning_rate: 5.0000e-04
Epoch 2/3
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - loss: 370.8620 - val_loss: 105.7917 - learning_rate: 5.0000e-04
Epoch 3/3
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - loss: 237.3216 - val_loss: 69.3325 - learning_rate: 5.0000e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step
[np.float64(0.08517633307644036), np.float64(0.15231072579708155), np.float64(0.2247124700300189), np.float64(0.23951225806519605), np.float64(0.1889106973678733)]
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Overall: 0.17812449686732168
Blend: 0.38754734322874024


Because for this dataset we know the ground truth, we can compute the correlation score for test data:

In [31]:
from senkin_tmp_cite_pred.metrics import correlation_score

In [32]:
correlation_score(adata_prot[test_cell_ids].layers["dsb"][:, :3], test_preds)

np.float64(0.717635762821232)